# Data Generation Needed for ACR/SWI Paper

In [2]:
#-------------------------- Standard Imports --------------------------#
%reload_ext autoreload
%autoreload 2
import kdephys as kde
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import acr
import os
plt.style.use('fast')
plt.style.use('/home/kdriessen/gh_master/kdephys/kdephys/plot/acr_plots.mplstyle')

# ---------------------------- EXTRAS --------------------------------#
from acr.utils import swi_subs_exps, sub_probe_locations, sub_exp_types


/home/kdriessen/miniconda3/envs/acr/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def check_df(subject, exp, path):
    if os.path.exists(path) == False:
        already_added = False
        df = pd.DataFrame()
        return df, already_added
    df = pd.read_csv(path)
    for col in df.columns:
        if "Unnamed" in col:
            del df[col]
    already_added = not df.loc[(df.subject==subject)&(df.exp==exp)].empty
    return df, already_added

In [4]:
# IMPORTANT PARAMETERS
# --------------------
REBOUND_LENGTH = '1h'
REL_STATE = 'NREM'

# Bandpower - During Sleep Rebound

In [5]:
# NOTES:
# ------

# We take as the start of the rebound the first time that NREM occurs after the stimulation has ended.
# Then we simply add the variable REBOUND_LENGTH to that start time to get the end of the rebound.
# So, subjects may have differing amounts of NREM in this initial 1 hr following the start of the sleep rebound. 

In [6]:
def get_rebound_df(subject, exp, reprocess_existing=False, save=True):
    reb_df, already_added = check_df(subject, exp, 'data/reb_df.csv')
    if already_added == True:
        if reprocess_existing:
            reb_df = reb_df.loc[(reb_df.subject!=subject)|(reb_df.exp!=exp)]
        else:
            return
    
    # load some basic information, and the hypnogram
    h = acr.io.load_hypno_full_exp(subject, exp)
    si = acr.info_pipeline.load_subject_info(subject)
    params = acr.info_pipeline.subject_params(subject)
    stores = params['time_stores']
    sort_ids = [f'{exp}-{store}' for store in stores]
    recordings = acr.info_pipeline.get_exp_recs(subject, exp)
    
    # load some temporal information about the rebound, baseline, sd, etc. 
    stim_start, stim_end = acr.stim.stim_bookends(subject, exp)
    reb_start = h.hts(stim_end-pd.Timedelta('15min'), stim_end+pd.Timedelta('1h')).st('NREM').iloc[0].start_time
    if reb_start < stim_end:
        stim_end_hypno = h.loc[(h.start_time<stim_end)&(h.end_time>stim_end)] # if stim time is in the middle of a nrem bout, then it can be the start of the rebound
        if stim_end_hypno.state.values[0] == 'NREM':
            reb_start = stim_end
        else:
            raise ValueError('Rebound start time is before stim end time, need to inspect')

    assert reb_start >= stim_end, 'Rebound start time is before stim end time'

    bl_start_actual = si["rec_times"][f'{exp}-bl']["start"]
    bl_day = bl_start_actual.split("T")[0]
    bl_start = pd.Timestamp(bl_day + "T09:00:00")

    if f'{exp}-sd' in si['rec_times'].keys():
        sd_rec = f'{exp}-sd'
        sd_end = pd.Timestamp(si['rec_times'][sd_rec]['end'])
    else:
        sd_rec = exp
        sd_end = stim_start
    sd_start_actual = pd.Timestamp(si['rec_times'][sd_rec]['start'])
    sd_day = si['rec_times'][sd_rec]['start'].split("T")[0]
    sd_start = pd.Timestamp(sd_day + "T09:00:00")
    
    # Load the BANDPOWER DATA
    #-------------------------------
    bp = acr.io.load_concat_bandpower(subject, recordings, stores, hypno=True);
    bp_rel = kde.xr.utils.rel_by_store(bp, state=REL_STATE, t1=None, t2=None);

    # Get the rebound values:
    rebound_dur = pd.Timedelta(REBOUND_LENGTH)
    reb_bp = bp_rel.ts(reb_start, reb_start+rebound_dur) 
    bp_df = kde.xr.spectral.bp_melt(reb_bp.to_dataframe().reset_index()) #convert to dataframe
    bp_df['region'] = sub_probe_locations[subject]
    bp_df['exp_type'] = sub_exp_types[subject]
    bp_df['subject'] = subject
    bp_df['exp'] = exp
    reb_df = pd.concat([reb_df, bp_df])
    if save:
        reb_df.to_csv('reb_df.csv') 
    return

# Bandpower - During Experimental Manipulation 

In [7]:
def get_stim_df(subject, exp, reprocess_existing=False, save=True):
    stim_df, already_added = check_df(subject, exp, 'data/stim_df.csv')
    if already_added == True:
        if reprocess_existing:
            stim_df = stim_df.loc[(stim_df.subject!=subject)|(stim_df.exp!=exp)]
        else:
            return
    
    # load some basic information, and the hypnogram
    h = acr.io.load_hypno_full_exp(subject, exp)
    si = acr.info_pipeline.load_subject_info(subject)
    params = acr.info_pipeline.subject_params(subject)
    stores = params['time_stores']
    sort_ids = [f'{exp}-{store}' for store in stores]
    recordings = acr.info_pipeline.get_exp_recs(subject, exp)
    
    # load some temporal information about the rebound, baseline, sd, etc. 
    stim_start, stim_end = acr.stim.stim_bookends(subject, exp)
    reb_start = h.hts(stim_end-pd.Timedelta('15min'), stim_end+pd.Timedelta('1h')).st('NREM').iloc[0].start_time
    if reb_start < stim_end:
        stim_end_hypno = h.loc[(h.start_time<stim_end)&(h.end_time>stim_end)] # if stim time is in the middle of a nrem bout, then it can be the start of the rebound
        if stim_end_hypno.state.values[0] == 'NREM':
            reb_start = stim_end
        else:
            raise ValueError('Rebound start time is before stim end time, need to inspect')

    assert reb_start >= stim_end, 'Rebound start time is before stim end time'

    bl_start_actual = si["rec_times"][f'{exp}-bl']["start"]
    bl_day = bl_start_actual.split("T")[0]
    bl_start = pd.Timestamp(bl_day + "T09:00:00")

    if f'{exp}-sd' in si['rec_times'].keys():
        sd_rec = f'{exp}-sd'
        sd_end = pd.Timestamp(si['rec_times'][sd_rec]['end'])
    else:
        sd_rec = exp
        sd_end = stim_start
    sd_start_actual = pd.Timestamp(si['rec_times'][sd_rec]['start'])
    sd_day = si['rec_times'][sd_rec]['start'].split("T")[0]
    sd_start = pd.Timestamp(sd_day + "T09:00:00")
    
    # Load the BANDPOWER DATA
    #-------------------------------
    bp = acr.io.load_concat_bandpower(subject, recordings, stores, hypno=True);
    bp_rel = kde.xr.utils.rel_by_store(bp, state=REL_STATE, t1=None, t2=None);

    # Get the SD values during the experimental manipulation:
    sd_bp = bp_rel.ts(stim_start, stim_end) 
    bp_df = kde.xr.spectral.bp_melt(sd_bp.to_dataframe().reset_index()) #convert to dataframe
    bp_df['region'] = sub_probe_locations[subject]
    bp_df['exp_type'] = sub_exp_types[subject]
    bp_df['subject'] = subject
    bp_df['exp'] = exp
    stim_df = pd.concat([stim_df, bp_df])
    if save:
        stim_df.to_csv('stim_df.csv') 
    return

# Generate Data

In [9]:
# Sleep Rebound Bandpower
# ------------------------
# Note that as currently configured, this does NOT reprocess subject-experiment combinations that are already in the dataframe. 
# If this needs to be done, call the function with reprocess_existing=True, for the particular subject-experiment combination

get_rebound_df('ACR_20', 'swi')